<a href="https://colab.research.google.com/github/Pham-Canh-An-Tien/data-mining1/blob/master/naive_bayes_text_clasifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tien Pham
1001909301

**This is a notebook for text classification using Naive Bayes Classifier**

In [2]:
from sklearn.model_selection import train_test_split
import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
! pip install kaggle
from google.colab import files
# Select the kaggle.json file
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download gaveshjain/ford-sentence-classifiaction-dataset
! unzip ford-sentence-classifiaction-dataset.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
ford-sentence-classifiaction-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ford-sentence-classifiaction-dataset.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test_data.csv           
replace train_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: all
error:  invalid response [all]
replace train_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train_data.csv          


In [4]:
train_dataset = pd.read_csv('train_data.csv')
test_dataset = pd.read_csv('test_data.csv')

print(train_dataset.shape)
print(test_dataset.shape)
train_dataset.dropna(axis = 0, how ='any', inplace = True)
test_dataset.dropna(axis = 0, how ='any', inplace = True)

train_dataset['Type'].value_counts()



(60115, 4)
(15029, 3)


Responsibility    15257
Requirement       13878
SoftSkill          9416
Experience         9078
Skill              6833
Education          4540
Name: Type, dtype: int64

In [ ]:
def my_func(row):
    if 'RES' in row['Sentence_id']:
        val = 'Responsibility'
    elif 'REQ' in row['Sentence_id']:
        val = 'Requirement'
    elif 'SKL' in row['Sentence_id']:
        val = 'Skill'
    elif 'SSK' in row['Sentence_id']:
        val = 'SoftSkill'
    elif 'EXP' in row['Sentence_id']:
        val = 'Experience'
    elif 'EDU' in row['Sentence_id']:
        val = 'Education'    
    return val

test_dataset['Type'] = test_dataset.apply(my_func, axis=1)

print(train_dataset.shape)
print(test_dataset.shape)
display(train_dataset['New_Sentence'])

In [6]:
merged_dataset = pd.concat([train_dataset,test_dataset])

def class_tokenize(row):
  if row['Type'] == 'Responsibility':
    rtv = 0
  elif row['Type'] == 'Requirement':
    rtv = 1
  elif row['Type'] == 'Skill':
    rtv = 2
  elif row['Type'] == 'SoftSkill':
    rtv = 3
  elif row['Type'] == 'Experience':
    rtv = 4
  elif row['Type'] == 'Education':
    rtv = 5
  return rtv

merged_dataset['Class'] = merged_dataset.apply(class_tokenize, axis=1)

merged_dataset.head(20)

,Unnamed: 0,Sentence_id,New_Sentence,Type,Class
0,0,GERRES15609,Author and/or Review architecture/design and o...,Responsibility,0
1,1,PHERES15784,Should be able to develop custom dynamic shape...,Responsibility,0
2,2,GERREQ10457,Experience in working crosslly with a larger ...,Requirement,1
3,3,GERSKL27235,"Previous business experience, including but no...",Skill,2
4,4,HONSSK18415,Delivering fast and right the first time.,SoftSkill,3
5,5,INDRES9494,Track department expenses and capital budget .,Responsibility,0
6,6,HONRES51852,Meet performance metrics.,Responsibility,0
7,7,UAEREQ34683,YOU MUST HAVEBachelors degree in Engineering.,Requirement,1
8,8,UAEREQ4612,After COVID-19: Ability to travel to manufactu...,Requirement,1
9,9,INDEDU42721,Six Sigma qualification to at least Green Bel...,Education,5


In [7]:
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2
train_set,remain_set = train_test_split(merged_dataset, test_size = (val_ratio+test_ratio))
train_set,remain_set = train_test_split(merged_dataset, test_size = (val_ratio+test_ratio))
val_set,test_set = train_test_split(remain_set,test_size =test_ratio/(val_ratio+test_ratio))
X_train = train_set['New_Sentence'].values
Y_train =  train_set['Class'].values
X_dev = val_set['New_Sentence'].values
Y_dev =  val_set['Class'].values
X_test = test_set['New_Sentence'].values
Y_test =  test_set['Class'].values

print(len(X_train))


44250


In [8]:
import re
word_dict = dict()
vocab_list = []

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", 
  "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 
  'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 
  'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 
  'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 
  'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 
  'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
  'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 
  'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 
  'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
  'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 
  'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
  'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 
  "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]


# sentence = X_train[1]
# words = re.split(' |,|\.|\/',sentence)
# print(words)


def build_vocab(dataset):
  for sentence in dataset:
    words = re.split(' |,|\.|\/',sentence)
    for word in words:
      if word not in stop_words and word != '':
        if word not in vocab_list:
          vocab_list.append(word)
          word_dict[word] = 1
        else:
          word_dict[word]+= 1
build_vocab(X_train)
print(vocab_list)

['Minimum', '10', 'years', 'experience', 'leading', 'marketing', 'strategic', 'planning', 'growth', 'initiatives', 'must', 'possess', 'proven', 'analytical', 'implementation', 'skills', 'Drive', 'programs', 'within', 'Advanced', 'Material', 'group', 'enabling', 'product', 'process', 'adoption', 'new', 'development', 'sales', 'pipeline', 'Support', 'execution', 'Fraud', 'Risk', 'Assessment', 'well', 'tracking', 'monitoring', 'Key', 'Indicators', 'Influence', 'business', 'leaders', 'channel', 'representatives', 'project', 'program', 'managers', 'drive', 'operational', 'financial', 'efficiencies', 'Ability', 'collaborate', 'levels', 'seniority', 'C-level', 'Engineers;', 'across', 'industries', 'Inclusion', 'diversity', 'practice', 'Bachelors', 'degree', 'five', 'B2B', 'industrial', 'engineering', 'role', 'Deploy', 'latest', 'security', 'antivirus', 'updates', 'IAS', 'Experience', 'image', 'capture', 'APIs', 'using', 'AVCaptureSession', 'manipulation', 'Associate', 'technical', 'college', 

In [9]:
print(len(vocab_list))
print(word_dict)

29808
{'Minimum': 903, '10': 286, 'years': 5744, 'experience': 8961, 'leading': 411, 'marketing': 408, 'strategic': 391, 'planning': 510, 'growth': 326, 'initiatives': 280, 'must': 425, 'possess': 85, 'proven': 186, 'analytical': 484, 'implementation': 429, 'skills': 3439, 'Drive': 203, 'programs': 369, 'within': 886, 'Advanced': 252, 'Material': 56, 'group': 108, 'enabling': 26, 'product': 1132, 'process': 1116, 'adoption': 56, 'new': 782, 'development': 2089, 'sales': 844, 'pipeline': 73, 'Support': 437, 'execution': 362, 'Fraud': 3, 'Risk': 85, 'Assessment': 13, 'well': 471, 'tracking': 91, 'monitoring': 110, 'Key': 251, 'Indicators': 6, 'Influence': 16, 'business': 2704, 'leaders': 210, 'channel': 79, 'representatives': 18, 'project': 1520, 'program': 570, 'managers': 112, 'drive': 652, 'operational': 216, 'financial': 363, 'efficiencies': 21, 'Ability': 1995, 'collaborate': 94, 'levels': 378, 'seniority': 3, 'C-level': 20, 'Engineers;': 1, 'across': 694, 'industries': 84, 'Inclusi

In [10]:
vocab_cleaned = []

for word in vocab_list:
  if word_dict[word] <=5:
    word_dict.pop(word)
  else:
    vocab_cleaned.append(word)

print(len(vocab_cleaned))
print(len(word_dict))


6047
6047


In [11]:
print(word_dict)
word_ind = dict()
i = 0
for word in vocab_cleaned:
  word_ind[word] = i
  i+=1

print(word_ind)



{'Minimum': 903, '10': 286, 'years': 5744, 'experience': 8961, 'leading': 411, 'marketing': 408, 'strategic': 391, 'planning': 510, 'growth': 326, 'initiatives': 280, 'must': 425, 'possess': 85, 'proven': 186, 'analytical': 484, 'implementation': 429, 'skills': 3439, 'Drive': 203, 'programs': 369, 'within': 886, 'Advanced': 252, 'Material': 56, 'group': 108, 'enabling': 26, 'product': 1132, 'process': 1116, 'adoption': 56, 'new': 782, 'development': 2089, 'sales': 844, 'pipeline': 73, 'Support': 437, 'execution': 362, 'Risk': 85, 'Assessment': 13, 'well': 471, 'tracking': 91, 'monitoring': 110, 'Key': 251, 'Indicators': 6, 'Influence': 16, 'business': 2704, 'leaders': 210, 'channel': 79, 'representatives': 18, 'project': 1520, 'program': 570, 'managers': 112, 'drive': 652, 'operational': 216, 'financial': 363, 'efficiencies': 21, 'Ability': 1995, 'collaborate': 94, 'levels': 378, 'C-level': 20, 'across': 694, 'industries': 84, 'diversity': 19, 'practice': 79, 'Bachelors': 1265, 'degree

In [92]:
nfeatures = len(vocab_cleaned)
def word_extraction(data,label):
  feature_matrix = np.zeros([6,nfeatures])
  label_count = np.zeros(6)
  for i in range(len(data)):
    label_count[label[i]] += 1
    words = re.split(' |,|\.|\/',data[i])
    for word in words:
      if word in vocab_cleaned:
        feature_matrix[label[i],int(word_ind[word])] +=1
    
  return feature_matrix,label_count

training_features,label_count = word_extraction(X_train,Y_train)
print(training_features)



[[1.000e+00 5.000e+00 4.000e+00 ... 0.000e+00 2.000e+00 0.000e+00]
 [2.700e+01 6.000e+00 4.100e+01 ... 2.000e+00 2.000e+00 0.000e+00]
 [3.000e+00 1.200e+01 2.000e+01 ... 2.000e+00 2.000e+00 7.000e+00]
 [0.000e+00 4.000e+00 4.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [8.430e+02 2.570e+02 5.631e+03 ... 1.000e+00 0.000e+00 0.000e+00]
 [2.900e+01 2.000e+00 4.400e+01 ... 2.000e+00 0.000e+00 0.000e+00]]


[  27.    6.   41. 1577.   86.   61.   78.   83.   41.   34.  321.   55.
   51.  106.   89.  743.   11.   68.  207.   52.]


In [93]:
prob_dict = np.zeros([6, nfeatures])
for i in range(len(label_count)):
    for j in range(nfeatures):
      prob_dict[i,j] = (training_features[i,j])/(label_count[i])

In [94]:
label_count = label_count
prob_class = label_count/sum(label_count)
print(prob_class)


[0.25968362 0.23423729 0.11554802 0.15848588 0.15428249 0.07776271]


In [81]:
def feature_extraction(dataset):
  feature_matrix = np.zeros([len(dataset),nfeatures])
  for i in range(len(dataset)):
    words = re.split(' |,|\.|\/',dataset[i])
    for word in words:
      if word in vocab_cleaned:
        feature_matrix[i,int(word_ind[word])] +=1
  return feature_matrix
DevSet = feature_extraction(X_dev)

In [95]:
def scorecalculation(vector):
  score = np.ones(6)
  # print(score)
  for i in range(6):
    for j in range(len(vector)):
      if vector[j]>0:
        score[i] = score[i]*prob_dict[i,j]
      else:
        score[i] = score[i]*(1-prob_dict[i,j])
    score[i]*=prob_class[i]
  return score

def predict(dataset,label):
  prediction = []
  n_correct = 0
  for i in range(len(dataset)):
    vec = dataset[i]
    prob_c = scorecalculation(vec)
    rtv = 0
    maxprob = 0
    for i in range(len(prob_c)):
      if prob_c[i]>maxprob:
        rtv = i
        maxprob = prob_c[i]
    prediction.append(rtv)
    if rtv == label[i]:
      n_correct+=1
  return prediction, n_correct

ypred, nCorrect = predict(DevSet, Y_dev)
print(nCorrect)


3859


In [96]:
print("Performance = ",nCorrect/len(Y_dev))

Performance =  0.2616271186440678


Laplace Smoothing

In [105]:
prob_dict1 = np.zeros([6, nfeatures])
for i in range(len(label_count)):
    for j in range(nfeatures):
      prob_dict1[i,j] = (training_features[i,j]+1)/(label_count[i]+nfeatures)

label_count1 = label_count + nfeatures
prob_class1 = label_count1/sum(label_count1)
print(prob_class1)

[0.21777678 0.20379476 0.13857845 0.16217156 0.15986192 0.11781652]


In [106]:
def scorecalculation1(vector):
  score = np.ones(6)
  # print(score)
  for i in range(6):
    for j in range(len(vector)):
      if vector[j]>0:
        score[i] = score[i]*prob_dict1[i,j]
      else:
        score[i] = score[i]*(1-prob_dict1[i,j])
    score[i]*=prob_class1[i]
  return score

def predict1(dataset,label):
  prediction = []
  n_correct = 0
  for i in range(len(dataset)):
    vec = dataset[i]
    prob_c = scorecalculation1(vec)
    rtv = 0
    maxprob = 0
    for i in range(len(prob_c)):
      if prob_c[i]>maxprob:
        rtv = i
        maxprob = prob_c[i]
    prediction.append(rtv)
    if rtv == label[i]:
      n_correct+=1
  return prediction, n_correct

ypred1, nCorrect1 = predict1(DevSet, Y_dev)
print(nCorrect1)
print("Performance = ",nCorrect1/len(Y_dev))

3965
Performance =  0.2688135593220339


In [107]:
TestSet = feature_extraction(X_test)
ypred1, nCorrect1 = predict1(TestSet, Y_test)
print(nCorrect1)
print("Performance = ",nCorrect1/len(Y_test))

3950
Performance =  0.2677966101694915
